In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
import numpy as np
import seaborn as  sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.metrics import mean_squared_error, mean_absolute_error 
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from ast import literal_eval
import nltk
from nltk.corpus import wordnet 
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from surprise import SVD,Dataset, Reader
import warnings
warnings.filterwarnings('ignore')

simple recommender:

1. Basic recommendations using movie polularity and acclaims
2. No personalized recommendations
3. Sort and display to all general audience


In [3]:
ratings= pd.read_csv(r'/kaggle/input/the-movies-dataset/ratings.csv')
links_small= pd.read_csv(r'/kaggle/input/the-movies-dataset/links_small.csv')
credits= pd.read_csv(r'/kaggle/input/the-movies-dataset/credits.csv')
keywords= pd.read_csv(r'/kaggle/input/the-movies-dataset/keywords.csv')
metadata= pd.read_csv(r'/kaggle/input/the-movies-dataset/movies_metadata.csv')
ratings_small= pd.read_csv(r'/kaggle/input/the-movies-dataset/ratings_small.csv')
links= pd.read_csv(r'/kaggle/input/the-movies-dataset/links.csv')


In [4]:
metadata.head(2)

In [46]:
metadata.shape

In [5]:

metadata_full=metadata.copy()


In [48]:
# metadata=metadata_small.copy()
# 

In [49]:
metadata.isnull().sum()

In [50]:
metadata['genres'].iloc[0]

In [51]:
metadata['belongs_to_collection'].iloc[0]

In [34]:
metadata['genres'] = metadata['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [78]:
vote_counts=metadata[metadata['vote_count'].notnull()]['vote_count'].astype('int')
vote_avg=metadata[metadata['vote_average'].notnull()]['vote_average'].astype('int')

mean_vote=vote_avg.mean()
mean_vote

In [79]:
min_votes=vote_counts.quantile(0.95)
min_votes

In [60]:
metadata['year'] = pd.to_datetime(metadata['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [56]:
qualified_movies=metadata[(metadata['vote_count']>min_votes) & (metadata['vote_count'].notnull()) & (metadata['vote_average']>mean_vote) & (metadata['vote_average'].notnull())]#[['title','year','vote_count','vote_average','genre','popularity']]
qualified_movies=qualified_movies[['title','year','vote_count','vote_average','genres','popularity']]

In [57]:
qualified_movies['vote_count'] = qualified_movies['vote_count'].astype('int')
qualified_movies['vote_average'] = qualified_movies['vote_average'].astype('int')
qualified_movies.columns

In [58]:
qualified_movies.shape

In [74]:
# weighted_rating= ((v/(v+m)*R)+((m/(v+m))*C)

def weighted_average(X):
    v=X['vote_count']
    R=X['vote_average']
    return ((v/(v+min_votes) * R) + (min_votes/(v+min_votes) * mean_vote))

In [60]:
# qualified_movies['weighted_rating']=''

In [61]:
from tqdm import tqdm
tqdm.pandas()
qualified_movies['weighted_rating']=qualified_movies.progress_apply(weighted_average,axis=1)

In [62]:
qualified=qualified_movies.sort_values('weighted_rating',ascending=False)

In [63]:
qualified.head(20)

In [64]:
#reduce percentile to 85% instead of 95% to make more intesting.
#separate the groupoed genres and form a meta data for each genre for each movie. Yhis give more correlation and genre categories to moies
saml = metadata.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
saml.name = 'genre'
genre_metadata = metadata.drop('genres', axis=1).join(saml)

    

In [65]:
genre_metadata.shape

In [66]:
# saml_2=[genre[0] for genre in metadata['genres'] if len(genre)>0]

In [67]:
def simple_recomm(genre,percentile=0.85):
    df=genre_metadata[genre_metadata['genre']==genre]
    vote_counts=df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_avg=df[df['vote_average'].notnull()]['vote_average'].astype('int')
    c=vote_avg.mean()
    m=vote_counts.quantile(percentile)
    
    qualified_df=df[(df['vote_count']>m) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())]
    qualified_df=qualified_df[['title','year','vote_count','vote_average','popularity']]
    qualified_df['vote_count']=qualified_df['vote_count'].astype('int')
    qualified_df['vote_average']=qualified_df['vote_average'].astype('int')
    
    qualified_df['weighted_rating']=qualified_df.apply(weighted_average,axis=1)
    qualified_=qualified_df.sort_values('weighted_rating',ascending=False).head(200)
    
    return qualified_

In [68]:
simple_recomm('Romance').head(5)

In [69]:
simple_recomm('Action').head(5)

In [70]:
%time
simple_recomm('Adventure').head(5)

In [71]:
%time
simple_recomm('Crime').head(5)

Content based recommendation systems:

Limitations of prev recommendation:
1. same recommendation to every person
2. No personalization

Advantages of content based recomm:
1.  personalized systems
2. can vary from person to person

In [6]:
links=links[links['tmdbId'].notnull()]['tmdbId'].astype('int')

In [7]:
metadata = metadata.drop([19730, 29503, 35587])
metadata['id']=metadata['id'].astype('int')

In [74]:
filtered_metadata=metadata[metadata['id'].isin(links)]
filtered_metadata.shape

Build MRE using descriptions and tags


In [75]:
filtered_metadata.tagline=filtered_metadata.tagline.fillna('')
filtered_metadata['description']=filtered_metadata.overview + filtered_metadata.tagline
filtered_metadata['description']=filtered_metadata['description'].fillna('')


In [76]:
filtered_metadata_sam=filtered_metadata.sample(n=20000)

In [77]:
tf=TfidfVectorizer(stop_words='english',ngram_range=(1,2),min_df=0)
tf_matrix=tf.fit_transform(filtered_metadata_sam.description)

1. calculate dotproduct to calculate cosine similarity

2. his cosine similarity gives the similarity between movies from the descripton and over

In [78]:
cosine_sim = linear_kernel(tf_matrix, tf_matrix)

In [79]:
cosine_sim[10]

In [80]:
filtered_metadata_sam=filtered_metadata_sam.reset_index()
titles=filtered_metadata_sam['title']
indices=pd.Series(filtered_metadata_sam.index,index=filtered_metadata_sam['title'])



In [81]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]
    

In [82]:
get_recommendations('Success').head(10)

In [88]:
get_recommendations('Richard Pryor: Live in Concert').head(10)

In [92]:
get_recommendations('The Graduate').head(10)

1. The above MRS will give only recommendation related to prev movies.
2. Lets make it more interering by adding more features
3. Cast and crew 


In [8]:
credits['id']=credits['id'].astype('int')
keywords['id']=keywords['id'].astype('int')
metadata = metadata_full.drop([19730, 29503, 35587])
metadata['id']=metadata['id'].astype('int')

In [12]:
metadata=metadata.merge(credits,on='id')
metadata=metadata.merge(keywords,on='id')

In [13]:
metadata=metadata[metadata['id'].isin(links)]

In [14]:
metadata['cast']=metadata['cast'].apply(literal_eval)
metadata['crew']=metadata['crew'].apply(literal_eval)
metadata['keywords']=metadata['keywords'].apply(literal_eval)
metadata['cast_size']=metadata['cast'].apply(lambda x : len(x))
metadata['crew_size']=metadata['crew'].apply(lambda x : len(x))



In [15]:
#now lets get director anme from  crew data
def get_director_name(data):
    for i in data:
        if i['job']=='Director':
            return i['name']
    return np.nan

In [16]:
metadata['director']=metadata['crew'].apply(get_director_name)

In [17]:
metadata['cast'] = metadata['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
metadata['cast'] = metadata['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)

In [18]:
metadata['keywords'] = metadata['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [19]:
metadata['cast']=metadata['cast'].apply(lambda x : [str.lower(i.replace(' ','')) for i in x ])

In [20]:
metadata['director'] = metadata['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
metadata['director'] = metadata['director'].apply(lambda x: [x,x, x])

In [21]:
metadata.head(3)

In [22]:
meta_df = metadata.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
meta_df.name = 'keyword'

In [23]:
meta_df = meta_df.value_counts()
meta_df[:5]

In [24]:
meta_df=meta_df[meta_df>1]

In [28]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in meta_df:
            words.append(i)
    return words

In [29]:
metadata['keywords']

In [31]:
stemmer = SnowballStemmer('english')
metadata['keywords'] = metadata['keywords'].apply(filter_keywords)
metadata['keywords'] = metadata['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
metadata['keywords'] = metadata['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [36]:
metadata['soup'] = metadata['keywords'] + metadata['cast'] + metadata['director'] + metadata['genres']
metadata['soup'] = metadata['soup'].apply(lambda x: ' '.join(x))

In [63]:
md_1=metadata[:11657]
md_2=metadata[11657:23314]
md_3=metadata[23314:34971]
md_4=metadata[34971:]

In [64]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix_1 = count.fit_transform(md_1['soup'])

In [65]:
count_matrix_3 = count.fit_transform(md_3['soup'])

In [66]:
count_matrix_2 = count.fit_transform(md_2['soup'])

In [67]:
count_matrix_4 = count.fit_transform(md_4['soup'])

In [68]:
cosine_sim1 = cosine_similarity(count_matrix_1, count_matrix_1)

In [69]:
cosine_sim2 = cosine_similarity(count_matrix_2, count_matrix_2)


In [70]:
cosine_sim3 = cosine_similarity(count_matrix_3, count_matrix_3)

In [71]:
cosine_sim4 = cosine_similarity(count_matrix_4, count_matrix_4)

In [72]:
md_1 = md_1.reset_index()
titles_1 = md_1['title']
indices_1 = pd.Series(md_1.index, index=md_1['title'])

md_2 = md_2.reset_index()
titles_2 = md_2['title']
indices_2 = pd.Series(md_2.index, index=md_2['title'])

md_3 = md_3.reset_index()
titles_3 = md_3['title']
indices_3 = pd.Series(md_3.index, index=md_3['title'])

md_4 = md_4.reset_index()
titles_4 = md_4['title']
indices_4 = pd.Series(md_4.index, index=md_4['title'])

In [80]:
def improved_recommendations(title,cosine_sim,smd):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year']]
    vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(0.60)
    qualified = movies[(movies['vote_count'] >= m) & (movies['vote_count'].notnull()) & (movies['vote_average'].notnull())]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    qualified['wr'] = qualified.apply(weighted_average, axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(10)
    return qualified

In [81]:
improved_recommendations('Mean Girls',cosine_sim2,md_2)

In [94]:
improved_recommendations('Toy Story',cosine_sim4,md_4)

In [103]:
improved_recommendations('Robin Hood',cosine_sim4,md_4)

In [118]:
# improved_recommendations('Inception',cosine_sim1,md_3)

In [119]:
reader = Reader()

In [121]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
data.split(n_folds=5)

In [ ]:
svd = SVD()
evaluate(svd, data, measures=['RMSE', 'MAE'])